<a href="https://colab.research.google.com/github/akatwinkle/bab_bab/blob/main/%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'
url


'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'

In [1]:
response = requests,get("http://openapi.seoul.go.kr:8088/sample/xml/energyUseDataSummaryInfo/1/5/%20/%20/")
response.status_code


NameError: name 'requests' is not defined

In [22]:
import requests
import json

url = 'http://openapi.seoul.go.kr:8088/4a48554755796f6f36375856727974/json/energyUseDataSummaryInfo/1/5/2015/01'
api_key = '4a48554755796f6f36375856727974'

params = {
    'KEY' : api_key,
    'TYPE' : 'json',
    'SERVICE' : 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '25'
}

def get_year_month():
    year_month = []
    for year in range(2015, 2024+1):
        for month in range(1,12+1):
            year_month.append(f'{year}{month:02d}')
    return year_month

resp = []

for year_month in get_year_month():

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()

        data = resp.json()


        print("API 호출 성공")
        print(data)

    except requests.exceptions.RequestException as e:
        print(f"API 호출 실패: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 실패: {e}")

API 호출 성공
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '학교', 'CNT': '1382', 'EUS': '134955565', 'EUS1': '128707423', 'EUS2': '145561511', 'ECO2_1': '-2178902', 'ECO2_2': '-923854.448', 'GUS': '9107197', 'GUS1': '8410968', 'GUS2': '10745416', 'GCO2_1': '-470995', 'GCO2_2': '-1055028.8', 'WUS': '2075819.2', 'WUS1': '2097433.

KeyboardInterrupt: 